In [1]:
import pandas as pd
import datetime
from pandas import Timedelta
import numpy as np

In [2]:
all_items_monthwise=pd.read_csv("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Stock&Monthwise_2022TD_Stock+Sales.csv")

C:\Users\David.Wagura\AppData\Local\Temp\ipykernel_9332\3801169709.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_items_monthwise=pd.read_csv("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Stock&Monthwise_2022TD_Stock+Sales.csv")


In [3]:
all_items=pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\All Item Master Data.xlsx")

In [4]:
full_items2023=pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Full Item List 15_02.xlsx")

In [5]:
Instock_import = pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Stock&Monthwise.xlsx")

In [6]:
wholesale_list_of_items = pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Wholesale Line.xlsx")

In [7]:
nw = all_items[['Item No.', 'Item Description', 'Existing Item Code']].copy()

In [8]:
nw['Old Codes']=nw['Existing Item Code'].astype(str).str.split('.', expand=True)[0]
nw.drop ('Existing Item Code', inplace=True, axis=1)

In [9]:
nw.rename(columns={'Item No.': 'New Item No.', 'Old Codes': 'Item No.'}, inplace=True)

In [10]:
nw2=nw[~nw.isin({'Item No.':['nan']})['Item No.']]
olditems_newitems_dict=dict (nw2[['Item No.', 'New Item No.']].values)

In [11]:
full_items2023['Item No.'] = full_items2023 ['Item No.'].astype(str)

In [12]:
full_items2023['Item No.']= full_items2023['Item No.'].apply(lambda x: olditems_newitems_dict[x] if x in olditems_newitems_dict else x)

In [13]:
date_created = full_items2023 [['Item No.', 'Date Created']].copy()

In [14]:
date_created['PD_Date'] = date_created['Date Created'].apply(lambda x: pd.to_datetime(x, format="%d/%m/%Y"))

In [15]:
date_created['Todays_Date'] =pd.to_datetime(datetime.datetime.now().date())

In [16]:
date_created

,Item No.,Date Created,PD_Date,Todays_Date
0,14400410001,18/06/2014,2014-06-18,2024-06-10
1,14400410003,18/06/2014,2014-06-18,2024-06-10
2,14400410004,18/06/2014,2014-06-18,2024-06-10
3,14400410007,18/06/2014,2014-06-18,2024-06-10
4,14400770005,18/06/2014,2014-06-18,2024-06-10
...,...,...,...,...
38112,157000187,09/12/2015,2015-12-09,2024-06-10
38113,163000007,02/08/2017,2017-08-02,2024-06-10
38114,175000010,18/06/2014,2014-06-18,2024-06-10
38115,160000139,11/05/2017,2017-05-11,2024-06-10


In [17]:
date_created['Tenure'] = date_created['Todays_Date'] - date_created['PD_Date']

In [18]:
# date_created.to_excel('ItemTenure.xlsx', index=False)

In [19]:
all_items_monthwise['Item No.'] = all_items_monthwise ['Item No.'].astype(str)

In [20]:
all_stores=all_items_monthwise ['Store/Location Name'].unique()

In [21]:
def item_rank(x):
    if x.Cumulative_Sum <= 0.8:
        return 'Top 80% Seller'
    elif x.Cumulative_Sum> 0.8:
        return 'Bottom 20% Seller'

In [22]:
def Rationalize(x):
    days_threshold = Timedelta('180 days')
    if x.Tenure>days_threshold and x.Sales<100000:
        return 'Rationalize'
    elif x.Tenure>days_threshold and x.Sales>=100000 and x.Sales<500000 :
        return 'At Risk'
    else:
        return 'Safe'

In [23]:
# shop[[i for i in shop.columns if 'sales' in i]].loc[:, 'month32023sales':].head(3)

In [26]:
shop_sales_empty_list=[]

In [27]:
for i in all_stores:
    shop=all_items_monthwise[all_items_monthwise['Store/Location Name']== i]
    shop_sales=pd.concat([shop.iloc[:, 0:8], shop[[i for i in shop.columns if 'sales' in i]].loc[:, 'month32023sales':].sum(axis=1)], axis=1)
    shop_sales.rename(columns={0: 'Sales'}, inplace=True)
    shop_sales=shop_sales.sort_values('Sales', ascending=False)
    total_sales=shop_sales.Sales.sum()
    
#     proportion_of_sales=shop_sales.Sales/total_sales
#     shop_sales['Proprtion'] = proportion_of_sales
#     cumulative_sum_proportion=shop_sales.Proprtion.cumsum()
#     shop_sales['Cumulative']=cumulative_sum_proportion
    
#     item_selling_status=shop_sales.apply(item_rank, axis=1)
#     shop_sales['Item_selling_status']=item_selling_status
#     status_table_only=shop_sales[['Item No.', 'Brand', 'Item Description', 'Store/Location Name', 'Item_selling_status']]
    status_table_only=shop_sales[['Item No.', 'Item Description', 'Brand', 'Store/Location Name', 'Sales']]
    

    shop_sales_empty_list.append(status_table_only)

In [28]:
trs=pd.concat(shop_sales_empty_list)

In [29]:
trs2 = pd.merge(left=trs, right = date_created[['Item No.','Tenure']], how='left', on=['Item No.'])

In [30]:
current_stock=Instock_import[['Item No.', 'Store/Location Name','InStock']].copy()

In [31]:
main_whse_instock=current_stock[current_stock['Store/Location Name'] == 'MAIN WAREHOUSE'].copy()

In [32]:
main_whse_instock.drop('Store/Location Name', axis=1, inplace=True)

In [33]:
main_whse = trs2[trs2['Store/Location Name'] == 'MAIN WAREHOUSE'].copy()

In [34]:
main_whse_with_instock=pd.merge(left=main_whse, right=main_whse_instock, on='Item No.', how='left')

In [35]:
comb=[]

In [36]:
for i in main_whse.Brand.unique():   
    tt=main_whse_with_instock[main_whse_with_instock['Brand'] == i].copy()
    tt_sales_proportion=main_whse_with_instock[main_whse_with_instock['Brand'] == i].Sales/main_whse_with_instock[main_whse_with_instock['Brand'] == i].Sales.sum()
    tt ['Sales_Sum_Proportion'] = tt_sales_proportion
    tt['Cumulative_Sum'] = tt.Sales_Sum_Proportion.cumsum()
    tt ['Selling_Status'] = tt.apply(item_rank, axis=1)
    tt['Rationalize_Status'] = tt.apply(Rationalize, axis=1)
    comb.append(tt)
    

In [37]:
final_table=pd.concat(comb)

In [38]:
final_table['InStock'].fillna(0, inplace=True)

In [39]:
wholesale_items =wholesale_list_of_items.copy()

In [40]:
wholesale_items

,#,Item No.,Item Description,In Stock,Bar Code,Item Group,Manufacturer,Inventory UoM,Last Eval. Price,Last Purchase Price,...,Exports Ex. VAT,Hotels Inc VAT,Key Account Inc VAT,Naivas Wholesale Inc VAT,Carrefour Inc VAT,Chandarana Inc VAT,QuickMart Inc VAT,Shoprite Inc VAT,Rubis Energy Inc VAT,Mega Wholesaler Inc VAT
0,1,100000036,DR BRON M SOAP ORG LIQ BABY MILD 8OZ/237ML OLBA08,119.0,18787772089,Baby Care,DR BRONNER'S,EACH,NaN,1188.26,...,1292.24,1199.2,1124.25,1124.25,1379.08,1124.25,1139.24,1199.2,1274.15,1499
1,2,100000037,ECOVER DELICATE DET. WOOL & SILK LAUNDRY 750ML...,NaN,5412533419566,Baby Care,ECOVER,EACH,NaN,NaN,...,NaN,799.2,749.25,749.25,919.08,749.25,759.24,799.2,849.15,999
2,3,100000038,ECOVER LAUNDRY LIQUID UNIVERSAL 1LTR#4005379,99.0,5412533420302,Baby Care,ECOVER,EACH,NaN,2060.25,...,2240.52,2079.2,1949.25,1949.25,2391.08,1949.25,1975.24,2079.2,2209.15,2599
3,4,101000037,ELLA'S KITCHEN ORG APPLES+BANANAS 120G#EK016,114.0,5060107330276,Baby foods,ELLA'S KITCHEN,EACH,NaN,193.90,...,430.17,399.2,374.25,374.25,459.08,374.25,379.24,399.2,424.15,499
4,5,101000038,ELLA'S KITCHEN ORG BANANA APPLE + BLUEBERRY BA...,131.0,5060503502260,Baby foods,ELLA'S KITCHEN,EACH,NaN,395.56,...,NaN,399.2,374.25,374.25,459.08,374.25,379.24,399.2,424.15,499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,1140,175000072,DIABLO SF MINT & CREAM SWEETS 75G #SWT-075-MNT...,74.0,5060309490242,Sugar free,DIABLO,EACH,NaN,232.97,...,516.38,479.2,449.25,449.25,551.08,449.25,455.24,479.2,509.15,599
1140,1141,175000089,DIABLO SF GUMMY BEARS 75G #05006AA,31.0,5060309490761,Sugar free,DIABLO,EACH,NaN,256.14,...,NaN,439.2,411.75,411.75,505.08,411.75,417.24,439.2,466.65,549
1141,1142,175000090,DIABLO SF COLA BOTTLES 75G #05007AA,399.0,5060309490778,Sugar free,DIABLO,EACH,NaN,395.56,...,NaN,439.2,411.75,411.75,505.08,411.75,417.24,439.2,466.65,549
1142,1143,175000091,DIABLO SF GUMMY DROPS 75G #05008AA,309.0,5060309490785,Sugar free,DIABLO,EACH,NaN,264.10,...,NaN,439.2,411.75,411.75,505.08,411.75,417.24,439.2,466.65,549


In [41]:
wholesale_items['Type'] ='Wholesale'

In [42]:
wholesale_items2=wholesale_items[['Item No.', 'Type']].copy()

In [43]:
wholesale_items2['Item No.'] = wholesale_items2['Item No.'].astype(str)

In [44]:
final2=pd.merge(left=final_table, right = wholesale_items2, on=['Item No.'], how='left')

In [45]:
final3=final2[final2.Type=='Wholesale']

In [ ]:
final3.to_excel('Rationalization_Wholesale2.xlsx', index=False)

### TOP 80 BOTTOM 20

In [46]:
top80bottom20=pd.concat([all_items_monthwise.iloc[:, :6], all_items_monthwise.loc[:, 'month32023sales':]], axis=1)

In [47]:
top80bottom20['TotalSales'] = top80bottom20[[i for i in top80bottom20.columns if 'sales' in i]].fillna(0).sum(axis=1)

In [53]:
short=top80bottom20[['Item No.', 'Brand', 'Department', 'TotalSales']]

In [54]:
short2=short.groupby(['Item No.', 'Brand', 'Department']).sum()

In [55]:
short2.TotalSales.sum()

3766858398.91

In [56]:
short['TotalSales'].sum()

3766858398.909999

In [57]:
short2.reset_index(inplace=True)

In [58]:
# short2short2.TotalSales>0

In [59]:
short3=short2.sort_values(by='TotalSales', ascending =False).copy()

In [60]:
def item_rank_top80_bottom20(x):
    
    

SyntaxError: incomplete input (3741784543.py, line 3)

In [61]:
short3['Rank'] = short3['TotalSales'].rank(ascending=False)

In [62]:
item_desc=top80bottom20[['Item No.', 'Item Description']].drop_duplicates()

In [63]:
item_desc_dict=dict(zip(item_desc['Item No.'].values, item_desc['Item Description'].values))

In [64]:
short3['Top 80_Bottom 20']=short3['Rank'].apply(lambda x: 'Top 80' if x< 0.8*len(short3) else 'Bottom 20')

In [65]:
short3.insert(1, 'Item Description', short3['Item No.'].apply(lambda x: item_desc_dict[x]))

In [66]:
short3

,Item No.,Item Description,Brand,Department,TotalSales,Rank,Top 80_Bottom 20
4595,153000137,SALAMA WHITE OATS(JMBO/SFST) 1KG,SALAMA,Packed Cereals,88292425.99,1.0,Top 80
7635,164000019,M/HARVEST CORNFLAKES 500G (VPBAG),M/HARVEST CEREALS,M/harvest,52394960.37,2.0,Top 80
1561,143000008,ALPRO ALMOND UNSWEETENED 1LTR,ALPRO,Dairy Alternatives,41047758.50,3.0,Top 80
6046,159000554,NATURES PLUS COLLAGEN PEPTIDES 294G,NATURES PLUS,Supplements&Vitamins,31823721.85,4.0,Top 80
1560,143000007,ALPRO ALMOND ORIGINAL MILK 1LTR_#80005347,ALPRO,Dairy Alternatives,27537430.17,5.0,Top 80
...,...,...,...,...,...,...,...
6677,159001197,THOMPSON OMEGA 3 1000MG 100CT#THO19320,THOMPSON,Supplements&Vitamins,-30082.03,14540.0,Bottom 20
6682,159001202,THOMPSON VIT E 265MG(400 IU) 30CT#THO19610,THOMPSON,Supplements&Vitamins,-37601.92,14541.0,Bottom 20
4865,155000020,WHITE HOUSE ORG APPLE CIDER VINEGAR RENEW L-G-...,WHITE HOUSE,Processed Products,-47874.72,14542.0,Bottom 20
6676,159001196,THOMPSON MULT FORMULA FOR WOMEN 60CT#THO19546,THOMPSON,Supplements&Vitamins,-77700.25,14543.0,Bottom 20


In [67]:
short4=short3.copy()

In [68]:
short4.drop('Rank', axis=1, inplace=True)

In [69]:
short5=short4[~short4.isin({ 'Department': ['Services', 'Assets & Consumables', 'Fixed Assets & Sundr']})['Department']]

In [70]:
# short5.to_excel('Top80_April.xlsx', index=False)

In [72]:
positive=pd.read_excel("C:\\Users\\David.Wagura\\Desktop\\Automation Sandbox\\Positive Inventory\\Positive Inventory (Retail) 10.06.2024.xlsx")

In [73]:
positive2=positive.copy()

In [74]:
short5.head(3)

,Item No.,Item Description,Brand,Department,TotalSales,Top 80_Bottom 20
4595,153000137,SALAMA WHITE OATS(JMBO/SFST) 1KG,SALAMA,Packed Cereals,88292425.99,Top 80
7635,164000019,M/HARVEST CORNFLAKES 500G (VPBAG),M/HARVEST CEREALS,M/harvest,52394960.37,Top 80
1561,143000008,ALPRO ALMOND UNSWEETENED 1LTR,ALPRO,Dairy Alternatives,41047758.50,Top 80


In [75]:
cols_=['Item No.', 'Item Description', 'Department', 'Brand']+[i for i in positive2.columns if 'RETAIL' in i and 'OVERSTOCK' not in i and 'UPCOUNTRY' not in i]+ [i for i in positive2.columns if 'MAIN' in i]+ [i for i in positive2.columns if 'RECEIVING' in i]

In [76]:
instock_shops = positive2[cols_].loc[:, 'RETAIL NYALI': 'RETAIL KISUMU MEGA'].sum(axis=1)

In [77]:
count_of_spread=(positive2[cols_].loc[:, 'RETAIL NYALI': 'RETAIL KISUMU MEGA']>0).sum(axis=1)

In [78]:
positive2['Instock Retail Shops'] = instock_shops
positive2['Count Of Shops With Stocks'] = count_of_spread

In [79]:
positive3 = positive2[['Item No.', 'Item Description', 'Department', 'Brand', 'Instock Retail Shops', 'Count Of Shops With Stocks', 'RETAIL WAREHOUSE', 'MAIN WAREHOUSE', 'RECEIVING WAREHOUSE']].copy()

In [80]:
positive3.rename({'RETAIL WAREHOUSE': 'Retail Warehouse', 'MAIN WAREHOUSE': 'Main Warehouse', 'RECEIVING WAREHOUSE': 'Receiving Warehouse'}, axis=1, inplace=True)

In [81]:
positive3

,Item No.,Item Description,Department,Brand,Instock Retail Shops,Count Of Shops With Stocks,Retail Warehouse,Main Warehouse,Receiving Warehouse
0,137000434,ON AMINO ENERGY BLUE RASPBERRY 270G #1132800,Sports Nutrition,OPTIMUM NUTRITION,32.0,19,0.0,0.0,0.0
1,100000047,HIPP ORG FOAMING HANDWASH 250ML #C003945,Baby Care,HIPP,23.0,12,0.0,0.0,0.0
2,167001748,YW MIRROR #AKA4862,Giftware,YIWU,0.0,0,0.0,0.0,0.0
3,167002556,SIL 6X7CM BUDDHA CANDLEPOT W/LID #FR1121,Giftware,SIL,4.0,2,0.0,0.0,0.0
4,175000079,VIAHEALTH CHOC HAZEL SPREAD WITH STEVIA-NAS 35...,Sugar free,VIAHEALTH,44.0,19,35.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
9317,167003447,YW TRAY #KA6469,Giftware,ONEIN,6.0,1,0.0,0.0,0.0
9318,167002832,YW GOLD RIM CLEAR FLOWER VASE LARGE #KA-5741,Giftware,ONEIN,0.0,0,0.0,0.0,0.0
9319,159000383,IRWIN STEEL LIBIDO FOR MEN 75CT#IN056238,Supplements&Vitamins,IRWIN NATURALS,96.0,32,56.0,0.0,0.0
9320,151001527,SOMERSET CLASSIC MILK COTTON LUXURY HAND WASH ...,Natural Cosmetics,SOMERSET,38.0,15,2.0,0.0,0.0


In [82]:
positive3['Item No.'] = positive3 ['Item No.'].astype(str)

In [83]:
short6 = pd.merge (left = short5, right = positive3 [['Item No.', 'Instock Retail Shops', 'Count Of Shops With Stocks', 'Retail Warehouse', 'Main Warehouse', 'Receiving Warehouse']], how = 'left', on=['Item No.'])

In [84]:
short6.to_excel('Top 80 June.xlsx', index=False)

In [ ]:
short6